In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ROOT = "/content/drive/25-2-Machine-Learning-Onions/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import numpy as np

# ==============================
# 0. 기본 설정 & 데이터 로드
# ==============================
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

print("Loading datafile...")
with open("{ROOT}data/raw/mon_standard.pkl", "rb") as fi:
    data = pickle.load(fi)

# X_time : timestamp 시퀀스 (abs(c))
# X_dir  : 방향 시퀀스 (+1/-1)
# X_size : 고정 크기 512 기반 size 시퀀스 (+512/-512)
X_time = []
X_dir  = []
X_size = []
y = []

for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE  # 같은 사이트 = 같은 레이블

    for sample in data[i]:
        time_seq = []
        dir_seq  = []
        size_seq = []

        for c in sample:
            dr = 1 if c > 0 else -1     # 방향
            time_seq.append(abs(c))     # 시간 정보
            dir_seq.append(dr)          # 방향 정보
            size_seq.append(dr * 512.0) # 🔵 고정 크기 512 적용

        X_time.append(time_seq)
        X_dir.append(dir_seq)
        X_size.append(size_seq)
        y.append(label)

print("Total samples:", len(y))


Loading datafile...
Total samples: 19000


In [ ]:
# ==============================
# 1. IAT/방향/Burst 기반 33차원 feature
# ==============================
def extract_features(time_seq, dir_seq):
    """
    time_seq : list/array of timestamps        (예: [0.0, 0.2, 0.25, ...])
    dir_seq  : list/array of direction (+1/-1) (예: [ 1,  -1,   -1,  ...])

    반환: 길이 33의 feature 벡터
    """

    t = np.asarray(time_seq, dtype=float)
    d = np.sign(np.asarray(dir_seq, dtype=float))  # +1(송신), -1(수신)

    # 패킷이 너무 적을 때 방어적 처리
    if len(t) < 2:
        return np.zeros(33, dtype=float)

    # ---------- [A] IAT 기반 feature (5개) ----------
    iat = np.diff(t)
    mean_iat   = float(np.mean(iat))
    std_iat    = float(np.std(iat))
    median_iat = float(np.median(iat))
    q25_iat, q75_iat = np.quantile(iat, [0.25, 0.75])

    # ---------- [B] 세션 요약 feature (4개) ----------
    n_packets = float(len(t))
    num_out = float((d > 0).sum())
    num_in  = float((d < 0).sum())

    ratio_out     = num_out / n_packets          # 전체 중 outgoing 비율
    switch_count  = float(np.sum(np.diff(d) != 0))
    duration      = float(t[-1] - t[0])

    # ---------- [C] Burst 기반 feature (5개) ----------
    bursts = []
    current_dir = d[0]
    count = 1
    for val in d[1:]:
        if val == current_dir:
            count += 1
        else:
            bursts.append((current_dir, count))
            current_dir, count = val, 1
    bursts.append((current_dir, count))

    burst_lens = np.array([b[1] for b in bursts], dtype=float)
    burst_dirs = np.array([b[0] for b in bursts], dtype=float)

    burst_count    = float(len(burst_lens))
    burst_mean     = float(burst_lens.mean()) if len(burst_lens) else 0.0
    burst_std      = float(burst_lens.std())  if len(burst_lens) else 0.0
    burst_max      = float(burst_lens.max())  if len(burst_lens) else 0.0
    burst_ratio_in = float((burst_dirs < 0).mean()) if len(burst_dirs) else 0.0

    # ---------- [D] 추가 feature (5개) ----------
    frac_in = num_in / n_packets    # incoming 비율 (outgoing 비율은 ratio_out)

    # 첫 30개 패킷 기준 통계
    first_k = min(30, len(d))
    d_first = d[:first_k]
    in_first30  = float((d_first < 0).sum())
    out_first30 = float((d_first > 0).sum())

    # ---------- [E] 첫 30개 패킷 기준 확장 feature (14개) ----------

    # [E1] IAT (first 30)
    t_first = t[:first_k]
    if len(t_first) >= 2:
        iat_first = np.diff(t_first)
        mean_iat_first30   = float(np.mean(iat_first))
        std_iat_first30    = float(np.std(iat_first))
        median_iat_first30 = float(np.median(iat_first))
        q25_iat_first30, q75_iat_first30 = np.quantile(iat_first, [0.25, 0.75])
    else:
        mean_iat_first30 = std_iat_first30 = 0.0
        median_iat_first30 = q25_iat_first30 = q75_iat_first30 = 0.0

    # [E2] 세션 요약 (first 30, n_packets 제외)
    ratio_out_first30 = out_first30 / float(first_k)
    switch_count_first30 = float(np.sum(np.diff(d_first) != 0)) if first_k > 1 else 0.0
    duration_first30 = float(t_first[-1] - t_first[0]) if first_k > 1 else 0.0

    # [E3] Burst (first 30)
    bursts_first = []
    current_dir = d_first[0]
    count = 1
    for val in d_first[1:]:
        if val == current_dir:
            count += 1
        else:
            bursts_first.append((current_dir, count))
            current_dir, count = val, 1
    bursts_first.append((current_dir, count))

    burst_lens_first = np.array([b[1] for b in bursts_first], dtype=float)
    burst_dirs_first = np.array([b[0] for b in bursts_first], dtype=float)

    burst_count_first30    = float(len(burst_lens_first))
    burst_mean_first30     = float(burst_lens_first.mean()) if len(burst_lens_first) else 0.0
    burst_std_first30      = float(burst_lens_first.std())  if len(burst_lens_first) else 0.0
    burst_max_first30      = float(burst_lens_first.max())  if len(burst_lens_first) else 0.0
    burst_ratio_in_first30 = float((burst_dirs_first < 0).mean()) if len(burst_dirs_first) else 0.0

    # [E4] 방향 비율 (first 30)
    frac_in_first30 = in_first30 / float(first_k)

 # ---------- 최종 feature 벡터 (총 33개) ----------
    return np.array([
        # [A] IAT 5개 (전체)
        mean_iat, std_iat, median_iat, q25_iat, q75_iat,
        # [B] 세션 요약 4개 (전체)
        ratio_out, switch_count, duration, n_packets,
        # [C] Burst 5개 (전체)
        burst_count, burst_mean, burst_std, burst_max, burst_ratio_in,
        # [D] 추가 5개 (전체 + 기존 first30 count)
        num_in, num_out, frac_in, in_first30, out_first30,

        # [E1] IAT 5개 (first 30)
        mean_iat_first30, std_iat_first30, median_iat_first30,
        q25_iat_first30, q75_iat_first30,
        # [E2] 세션 요약 3개 (first 30)
        ratio_out_first30, switch_count_first30, duration_first30,
        # [E3] Burst 5개 (first 30)
        burst_count_first30, burst_mean_first30, burst_std_first30,
        burst_max_first30, burst_ratio_in_first30,
        # [E4] 방향 비율 1개 (first 30)
        frac_in_first30
    ], dtype=float)


In [ ]:
# ==============================
# 2. 모든 monitored 세션에 대해 feature 행렬 생성
# ==============================
X_feat_monitored = np.vstack([
    extract_features(t, d)
    for t, d in zip(X_time, X_dir)
])
y_monitored = np.asarray(y, dtype=int)

print("X_feat_monitored.shape:", X_feat_monitored.shape)  # (N_mon, 19)
print("y_monitored.shape:", y_monitored.shape)

# ==============================
# 3. feature 이름 정의
# ==============================
feature_names = [
    # [A] IAT (전체)
    "mean_iat", "std_iat", "median_iat", "q25_iat", "q75_iat",
    # [B] 세션 요약 (전체)
    "ratio_out", "switch_count", "duration", "n_packets",
    # [C] Burst (전체)
    "burst_count", "burst_mean", "burst_std", "burst_max", "burst_ratio_in",
    # [D] 추가 (전체 + first30 count)
    "num_in", "num_out", "frac_in", "in_first30", "out_first30",

    # [E1] IAT (first 30)
    "mean_iat_first30", "std_iat_first30", "median_iat_first30",
    "q25_iat_first30", "q75_iat_first30",
    # [E2] 세션 요약 (first 30)
    "ratio_out_first30", "switch_count_first30", "duration_first30",
    # [E3] Burst (first 30)
    "burst_count_first30", "burst_mean_first30", "burst_std_first30",
    "burst_max_first30", "burst_ratio_in_first30",
    # [E4] 방향 비율 (first 30)
    "frac_in_first30",
]
print("len(feature_names):", len(feature_names))


X_feat_monitored.shape: (19000, 33)
y_monitored.shape: (19000,)
len(feature_names): 33


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1️⃣ 먼저 train+val 과 test를 분리
X_train_val, X_test_close, y_train_val, y_test_close = train_test_split(
    X_feat_monitored, y, test_size=0.15, random_state=42, stratify=y
)

# 2️⃣ 다음으로 train 과 val 분리
val_size = 0.15 / (1 - 0.15)
X_train_close, X_val_close, y_train_close, y_val_close = train_test_split(
    X_train_val, y_train_val, test_size=val_size, random_state=42, stratify=y_train_val
)


# 3️⃣ 스케일링
scaler = StandardScaler()
X_train_close_scaled = scaler.fit_transform(X_train_close)
X_val_close_scaled  = scaler.transform(X_val_close)
X_test_close_scaled = scaler.transform(X_test_close)





In [ ]:
# ==== 공통: 저장/미리보기 ====
import pickle, numpy as np, pandas as pd

def save_pickle(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)

def load_pickle(path):
    with open(path, "rb") as f:
        return pickle.load(f)

def head_pickle(path, n=5):
    d = load_pickle(path)
    X, y, cols = d["X"], d["y"], d["feature_names"]
    df = pd.DataFrame(X, columns=cols)
    df["label"] = y
    print(f"=== {path} === shape={X.shape}, labels={y.shape}")
    display(df.head(n))
    return d



# ==== dtype 정리 (권장) ====
X_train_close_scaled = X_train_close_scaled.astype("float32")
X_val_close_scaled   = X_val_close_scaled.astype("float32")
X_test_close_scaled  = X_test_close_scaled.astype("float32")
y_train_close = np.asarray(y_train_close, dtype="int64")
y_val_close   = np.asarray(y_val_close,   dtype="int64")
y_test_close  = np.asarray(y_test_close,  dtype="int64")

# ==== 개별 파일 저장 (dict 포맷) ====
base = {"feature_names": feature_names, "scaler": scaler}
SAVE_DIR = "{ROOT}data/preprocessed/"

save_pickle(f"{SAVE_DIR}close_train_33.pkl", {**base, "X": X_train_close_scaled, "y": y_train_close})
save_pickle(f"{SAVE_DIR}close_val_33.pkl",   {**base, "X": X_val_close_scaled,   "y": y_val_close})
save_pickle(f"{SAVE_DIR}close_test_33.pkl",  {**base, "X": X_test_close_scaled,  "y": y_test_close})
print("Saved: close_train_33.pkl / close_val_33.pkl / close_test_33.pkl")

_ = head_pickle(f"{SAVE_DIR}close_train_33.pkl", n=5)
_ = head_pickle(f"{SAVE_DIR}close_val_33.pkl",   n=5)
_ = head_pickle(f"{SAVE_DIR}close_test_33.pkl",  n=5)

Saved: close_train_33.pkl / close_val_33.pkl / close_test_33.pkl
=== /content/drive/MyDrive/4-2 기계학습/close_train_33.pkl === shape=(13300, 33), labels=(13300,)


,mean_iat,std_iat,median_iat,q25_iat,q75_iat,ratio_out,switch_count,duration,n_packets,burst_count,...,ratio_out_first30,switch_count_first30,duration_first30,burst_count_first30,burst_mean_first30,burst_std_first30,burst_max_first30,burst_ratio_in_first30,frac_in_first30,label
0,-0.222113,-0.247032,-0.008671,0.0,-0.109138,-0.646333,-0.942113,-0.961234,-0.806580,-0.942113,...,-0.888314,-0.829908,-0.190928,-0.829908,0.790760,1.213141,1.286695,0.720568,0.888314,93
1,-0.239861,-0.017324,-0.008671,0.0,-0.109138,-0.906819,1.263483,1.614872,1.737267,1.263483,...,-0.888314,-0.829908,2.064045,-0.829908,0.790760,0.273217,0.083641,0.720568,0.888314,13
2,1.488485,0.521502,-0.008671,0.0,-0.109138,1.167571,-1.214409,1.491876,-1.161659,-1.214409,...,-1.616619,-1.788464,1.013715,-1.788464,2.344068,1.267887,0.484659,1.313120,1.616619,24
3,-0.212814,-0.174084,-0.008671,0.0,-0.109138,-0.643929,-0.031867,0.485806,0.213765,-0.031867,...,-0.888314,-0.829908,-0.464710,-0.829908,0.790760,0.768398,0.885677,0.720568,0.888314,5
4,-0.308809,-0.443894,-0.008671,0.0,-0.109138,-0.734781,0.108171,-1.151112,0.408918,0.108171,...,2.024909,0.128648,0.306859,0.128648,-0.284606,-0.918551,-0.718395,0.310340,-2.024909,21


=== /content/drive/MyDrive/4-2 기계학습/close_val_33.pkl === shape=(2850, 33), labels=(2850,)


,mean_iat,std_iat,median_iat,q25_iat,q75_iat,ratio_out,switch_count,duration,n_packets,burst_count,...,ratio_out_first30,switch_count_first30,duration_first30,burst_count_first30,burst_mean_first30,burst_std_first30,burst_max_first30,burst_ratio_in_first30,frac_in_first30,label
0,-0.214956,0.168599,-0.008671,0.0,-0.109138,-0.654720,0.185970,0.401353,0.178849,0.185970,...,0.568297,1.087204,-0.111282,1.087204,-1.073209,-1.217435,-1.520432,0.009506,-0.568297,35
1,-0.020215,0.522437,-0.008671,0.0,-0.109138,0.473165,-0.700937,0.298196,-0.781640,-0.700937,...,-0.160008,-0.350630,-0.290485,-0.350630,0.208270,1.020009,1.286695,-1.945914,0.160008,60
2,-0.213171,-0.418527,-0.008671,0.0,-0.109138,0.304738,-0.942113,-1.101234,-0.947489,-0.942113,...,0.568297,0.128648,-0.524445,0.128648,-0.284606,-0.258376,0.083641,0.310340,-0.568297,58
3,-0.164183,-0.122699,-0.008671,0.0,-0.109138,-0.497820,0.294888,1.522484,0.308224,0.294888,...,-0.888314,-0.829908,-0.872895,-0.829908,0.790760,0.768398,0.885677,0.720568,0.888314,9
4,-0.195238,-0.383007,-0.008671,0.0,-0.109138,0.214538,-0.934333,-1.026983,-0.947801,-0.934333,...,-0.160008,0.128648,-0.111282,0.128648,-0.284606,-0.179903,0.083641,0.310340,0.160008,58


=== /content/drive/MyDrive/4-2 기계학습/close_test_33.pkl === shape=(2850, 33), labels=(2850,)


,mean_iat,std_iat,median_iat,q25_iat,q75_iat,ratio_out,switch_count,duration,n_packets,burst_count,...,ratio_out_first30,switch_count_first30,duration_first30,burst_count_first30,burst_mean_first30,burst_std_first30,burst_max_first30,burst_ratio_in_first30,frac_in_first30,label
0,-0.100425,-0.101335,-0.008671,0.0,-0.109138,0.425576,-0.560899,0.097548,-0.673776,-0.560899,...,-0.888314,-0.829908,-0.902762,-0.829908,0.790760,0.107655,-0.317377,0.720568,0.888314,60
1,-0.126123,0.100509,-0.008671,0.0,-0.109138,-0.370599,-0.004638,1.446532,-0.024098,-0.004638,...,0.568297,0.607926,1.003760,0.607926,-0.707072,-0.213989,0.083641,-1.945914,-0.568297,83
2,-0.221186,-0.149585,-0.008671,0.0,-0.109138,-0.134073,1.096215,1.294630,0.967255,1.096215,...,0.568297,2.045760,0.545796,2.045760,-1.676257,-1.677687,-1.520432,-0.220543,-0.568297,46
3,2.784585,0.637580,-0.008671,0.0,10.576226,4.894014,-1.331107,-1.020748,-1.292592,-1.331107,...,2.024909,1.087204,1.815151,1.087204,-1.073209,-0.987815,-0.718395,0.009506,-2.024909,82
4,-0.221552,-0.432441,-0.008671,0.0,-0.109138,-0.368196,-1.237748,-1.382933,-1.142642,-1.237748,...,-0.888314,-0.829908,-0.857961,-0.829908,0.790760,0.470765,0.083641,0.720568,0.888314,70


In [ ]:
y_mon_train_bin = np.ones(len(y_train_close), dtype=int)   # (N_train,)
y_mon_val_bin = np.ones(len(y_val_close), dtype=int)
y_mon_test_bin = np.ones(len(y_test_close), dtype=int)

=================

In [ ]:
import pickle

TOTAL_URLS = 10000  # total number in the dataset

# Load 10,000 unmon pickle file
print("Loading datafile...")
with open("{ROOT}data/raw_data/unmon_standard10.pkl", 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    X1.append(time_seq)
    X2.append(size_seq)

print(len(X1)) # Print the length of X1

Loading datafile...
Total samples: 10000
10000


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# =========================================
# 1) 방향 정규화 (+1 / -1)
#    X2 : 각 세션별 방향/size 시퀀스 (ex: dr*512)
#         → sign만 쓰면 방향 시퀀스로 쓸 수 있음
# =========================================
X2_bin = [np.sign(np.asarray(seq, float)).astype(int).tolist() for seq in X2]

# =========================================
# 2) 모든 unmon 세션에 대해 monitored와 같은 feature (33개) 생성
#    X1 : time_seq 리스트, X2_bin : direction(+1/-1) 리스트
# =========================================
X_feat_unmon = np.vstack([
    extract_features(t, d)
    for t, d in zip(X1, X2_bin)
])
print("X_feat_unmon.shape:", X_feat_unmon.shape)  # (N_unmon, 33)

N_unmon = X_feat_unmon.shape[0]

# 라벨도 N_unmon에 맞춰 생성
y_unmon_true = -np.ones(N_unmon, dtype=int)
y_unmon_bin  = -np.ones(N_unmon, dtype=int)


X_feat_unmon.shape: (10000, 33)


In [ ]:
# =========================================
# 4) unmon 데이터 train/val/test split
# =========================================
X_unmon_train_val, X_unmon_test, y_unmon_train_val, y_unmon_test = train_test_split(
    X_feat_unmon,
    y_unmon_true,
    test_size=0.15,
    random_state=42,
)

val_size = 0.15 / (1 - 0.15)

X_unmon_train, X_unmon_val, y_unmon_train, y_unmon_val = train_test_split(
    X_unmon_train_val,
    y_unmon_train_val,
    test_size=val_size,
    random_state=42,
)

# binary 라벨 (형태만 맞춰주기)
y_unmon_train_bin = -np.ones_like(y_unmon_train, dtype=int)
y_unmon_val_bin   = -np.ones_like(y_unmon_val,   dtype=int)
y_unmon_test_bin  = -np.ones_like(y_unmon_test,  dtype=int)


# =========================================
# 5) open-world 데이터 구성 (monitored + unmonitored)
# =========================================
# 여기서 X_train_close, X_val_close, X_test_close 는
# "monitored feature 33차원짜리 (스케일링 전)" 이라고 가정
# y_mon_train_bin 등은 monitored에 대해 +1 / -1 라벨링 해둔 것

# 🔵 open-world train (raw)
X_train_open_raw = np.concatenate([X_train_close, X_unmon_train], axis=0)
y_train_open_true = np.concatenate([y_train_close, y_unmon_train])
y_train_open_bin  = np.concatenate([y_mon_train_bin, y_unmon_train_bin])

# 🟣 open-world val (raw)
X_val_open_raw = np.concatenate([X_val_close, X_unmon_val], axis=0)
y_val_open_true = np.concatenate([y_val_close, y_unmon_val])
y_val_open_bin  = np.concatenate([y_mon_val_bin, y_unmon_val_bin])

# 🔴 open-world test (raw)
X_test_open_raw = np.concatenate([X_test_close, X_unmon_test], axis=0)
y_test_open_true = np.concatenate([y_test_close, y_unmon_test])
y_test_open_bin  = np.concatenate([y_mon_test_bin, y_unmon_test_bin])


# =========================================
# 6) 스케일링
# =========================================
scaler = StandardScaler()
X_train_open = scaler.fit_transform(X_train_open_raw)
X_val_open   = scaler.transform(X_val_open_raw)
X_test_open  = scaler.transform(X_test_open_raw)


In [ ]:
save_pickle(f"{SAVE_DIR}open_train_33.pkl", {
    "X": X_train_open,
    "y_true": y_train_open_true,
    "y_binary": y_train_open_bin,
    "feature_names": feature_names,
    "scaler": scaler,   # 나중에 inverse_transform 등 가능
})

save_pickle(f"{SAVE_DIR}open_val_33.pkl", {
    "X": X_val_open,
    "y_true": y_val_open_true,
    "y_binary": y_val_open_bin,
    "feature_names": feature_names,
})

save_pickle(f"{SAVE_DIR}open_test_33.pkl", {
    "X": X_test_open,
    "y_true": y_test_open_true,
    "y_binary": y_test_open_bin,
    "feature_names": feature_names,
})


In [ ]:
import pickle
import pandas as pd

def head_pickle_2(path, n=5):
    with open(path, "rb") as f:
        data = pickle.load(f)

    X = data["X"]
    feature_names = data.get("feature_names", None)

    print(f"\n=== {path} === shape={X.shape}")

    if feature_names is not None and len(feature_names) == X.shape[1]:
        df = pd.DataFrame(X[:], columns=feature_names)
    else:
        df = pd.DataFrame(X[:])


    # 🔹 두 label을 모두 컬럼으로 추가
    if "y_true" in data:
        df["y_true"] = data["y_true"][:]
    if "y_binary" in data:
        df["y_binary"] = data["y_binary"][:]

    print(df.sample())
    return data


_ = head_pickle_2(f"{SAVE_DIR}open_train_33.pkl", n=5)
_ = head_pickle_2(f"{SAVE_DIR}open_val_33.pkl", n=5)
_ = head_pickle_2(f"{SAVE_DIR}open_test_33.pkl", n=100)



=== /content/drive/MyDrive/4-2 기계학습/open_train_33.pkl === shape=(20300, 33)
       mean_iat   std_iat  median_iat  q25_iat   q75_iat  ratio_out  \
10420 -0.215137 -0.299119   -0.013406      0.0 -0.138112   1.125719   

       switch_count  duration  n_packets  burst_count  ...  \
10420       1.73946  1.483396   0.693835      1.73946  ...   

       switch_count_first30  duration_first30  burst_count_first30  \
10420             -0.915252         -0.325114            -0.915252   

       burst_mean_first30  burst_std_first30  burst_max_first30  \
10420            0.758361           0.637668           0.666321   

       burst_ratio_in_first30  frac_in_first30  y_true  y_binary  
10420                0.798524         0.824874      89         1  

[1 rows x 35 columns]

=== /content/drive/MyDrive/4-2 기계학습/open_val_33.pkl === shape=(4350, 33)
      mean_iat   std_iat  median_iat  q25_iat   q75_iat  ratio_out  \
1667    0.5534  2.936251   -0.013406      0.0 -0.138112  -0.470018   

      s